<a href="https://colab.research.google.com/github/zlwmxkdla/graduation_capston/blob/main/age_add_webcam_captured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

#신경망 가중치와 결과의 무작위성을 줄이기 위해 랜덤 시드를 설정
#결과를 완전히 재현하는 것은 여전히 어려울 수 있음

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# combined_faces 압축 파일 풀기

combined_faces_zip_path = "/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/input/combined_faces.zip"
extracted_path='/content/combined_faces'
with ZipFile(combined_faces_zip_path, 'r') as myzip:
    myzip.extractall(extracted_path)
    print('Done unzipping combined_faces.zip')

Done unzipping combined_faces.zip


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# 기존에 학습한 모델을 불러옵니다.
model = load_model('/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/output/your_updated_model_2.h5')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 100, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 50, 50, 128)       0         
 g2D)                                                   

엑셀파일 만들기!!

In [ ]:
ages = []

path = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/webcam_captured'
i = 0

for img_filename in os.listdir(path):
    i = i + 1
    age_gender_filename = img_filename.split("_")
     # Ensure that the file is an image file
    if not img_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    ages.append(int(age_gender_filename[0]))  # 이미지 파일 이름에서 나이 정보 추출

print(ages)


[65, 65, 65, 65, 65, 65, 65, 3, 3, 3, 3, 3, 3, 23, 23, 23, 23, 23, 23, 23, 23, 21, 21, 21, 21, 21, 21, 21, 21, 24, 24, 24, 24, 24, 24, 24, 24, 26, 26, 26, 26, 26, 26, 26, 26, 21, 21, 21, 21, 21, 21, 21, 21, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 22, 24, 24, 24, 24, 24, 24, 24, 24, 26, 26, 26, 26, 26, 26, 26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 22, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 25, 25, 25, 25, 25, 25, 25, 25, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 33, 33, 33, 33, 33, 33, 33, 33, 28, 28, 28, 28, 28, 28, 28, 28, 34, 34, 34, 34, 34, 34, 34, 34, 36, 36, 36, 36, 36, 36, 36, 36, 33, 33, 33, 33, 33, 33, 33, 33, 28, 28, 28, 28, 28, 28, 2

In [ ]:
print(len(ages))

3096


In [ ]:
import os
import pandas as pd

# 사용자가 설정할 열 이름
column_name = 'filename'
column_name2 = 'age'

# 사진이 저장된 디렉토리 경로
photo_directory = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/webcam_captured'

# 디렉토리에서 파일 목록 가져오기
photo_files = [f for f in os.listdir(photo_directory) if os.path.isfile(os.path.join(photo_directory, f))]

# 파일 목록을 DataFrame으로 변환
data = {column_name: photo_files}
df = pd.DataFrame(data)

# 추가할 문자열
additional_string = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/webcam_captured/'

# 특정 열에 같은 문자열 추가
df[column_name] =additional_string+ df[column_name]

# age 관련 열 추가
df[column_name2] = ages

# DataFrame을 엑셀 파일로 저장
csv_filename = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/webcam_captured.csv'
df.to_csv(csv_filename, index=False)

print(f"엑셀 파일이 생성되었습니다: {csv_filename}")


엑셀 파일이 생성되었습니다: /content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/webcam_captured.csv


In [ ]:
age_counts = df['age'].value_counts()

print("Age Distribution:")
print(age_counts)

Age Distribution:
28    88
10    80
22    80
34    72
9     72
      ..
46     8
95     8
93     8
62     8
66     8
Name: age, Length: 78, dtype: int64


In [ ]:
# 증강된 데이터셋 블러와서 텐서 만들기

train_aug_df = pd.read_csv("/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/webcam_captured.csv")
test_df = pd.read_csv("/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/input/images_filenames_labels_test_1.csv")

In [ ]:
# 나이 범주별 라벨링

def class_labels_reassign(age):

    if 1 <= age <= 5:
        return 0
    elif 6<= age <= 10:
        return 1
    elif 11 <= age <= 15:
        return 2
    elif 16 <= age <=20:
        return 3
    elif 21 <= age <= 26:
        return 4
    elif 27 <= age <= 32:
        return 5
    elif 33 <= age <= 39:
        return 6
    elif 40 <= age <= 48:
        return 7
    elif 49 <= age <= 56:
        return 8
    elif 57 <= age <= 65:
        return 9
    else:
        return 10

In [ ]:
train_aug_df['target'] = train_aug_df['age'].map(class_labels_reassign)
test_df['target'] = test_df['age'].map(class_labels_reassign)

In [ ]:
# Count of each unique label in the 'target' column for train_aug_df
train_label_counts = train_aug_df['target'].value_counts().sort_index()

# Count of each unique label in the 'target' column for test_df
test_label_counts = test_df['target'].value_counts().sort_index()

# Print the results
print("Train Set Label Distribution:")
print(train_label_counts)

print("\nTest Set Label Distribution:")
print(test_label_counts)


Train Set Label Distribution:
0     280
1     288
2     304
3     280
4     280
5     280
6     280
7     280
8     272
9     280
10    272
Name: target, dtype: int64

Test Set Label Distribution:
0     1402
1      499
2      322
3      521
4     1780
5     1379
6     1059
7      797
8      851
9      627
10     809
Name: target, dtype: int64


In [ ]:
train_aug_df.head


<bound method NDFrame.head of                                                filename  age  target
0     /content/drive/MyDrive/GraduationCapston/Forei...   65       9
1     /content/drive/MyDrive/GraduationCapston/Forei...   65       9
2     /content/drive/MyDrive/GraduationCapston/Forei...   65       9
3     /content/drive/MyDrive/GraduationCapston/Forei...   65       9
4     /content/drive/MyDrive/GraduationCapston/Forei...   65       9
...                                                 ...  ...     ...
3091  /content/drive/MyDrive/GraduationCapston/Forei...   76      10
3092  /content/drive/MyDrive/GraduationCapston/Forei...   72      10
3093  /content/drive/MyDrive/GraduationCapston/Forei...   80      10
3094  /content/drive/MyDrive/GraduationCapston/Forei...   72      10
3095  /content/drive/MyDrive/GraduationCapston/Forei...   80      10

[3096 rows x 3 columns]>

In [ ]:
# 증강된 훈련,검증 데이터셋들을 위해 파일 이름과 타겟 클래스 라벨 변경

train_aug_filenames_list = list(train_aug_df['filename'])
train_aug_labels_list = list(train_aug_df['target'])

test_filenames_list = list(test_df['filename'])
test_labels_list = list(test_df['target'])

In [ ]:
# 위에서 정의한 목록들을 사용하여 증강된(데이터를 늘린) 학습 및 테스트 데이터셋에 대한 TensorFlow 상수를 만들고, 파일 이름과 레이블을 나타내기

train_aug_filenames_tensor = tf.constant(train_aug_filenames_list)
train_aug_labels_tensor = tf.constant(train_aug_labels_list)

test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

In [ ]:
# 주어진 텐서로부터 이미지를 읽고, 이미지를 해독하며, 이미지 레이블 클래스를 원-핫 인코딩하는 함수 정의.
# tf.io.decode_jpeg에서 channels 매개변수를 3에서 1로 변경하면 출력 이미지가 RGB 색상이 아닌 그레이스케일로 변함

num_classes = 11

def _parse_function(filename, label):

    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)    # channels=1 그레이 스케일로 바꾸기 위해서, channels=3 RGB로 바꾸기 위해서.
    image_resized = tf.image.resize(image_decoded, [200, 200])
    #image_resized = tf.image.grayscale_to_rgb(image_resized)
    label = tf.one_hot(label, num_classes)

    return image_resized, label

In [ ]:
# 뉴런 네트워크를 위해 데이터셋 가져오기
# 위에서 정의한 텐서 벡터를 사용하여 데이터셋에서 이미지에 접근하고, 위에서 정의한 함수를 통해 이미지를 처리

train_aug_dataset = tf.data.Dataset.from_tensor_slices((train_aug_filenames_tensor, train_aug_labels_tensor))
train_aug_dataset = train_aug_dataset.map(_parse_function)
#train_aug_dataset = train_aug_dataset.repeat(3)
train_aug_dataset = train_aug_dataset.batch(512)    # 아래 model.fit()에서의 batch_size 하이퍼파라미터와 동일

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_tensor, test_labels_tensor))
test_dataset = test_dataset.map(_parse_function)
#test_dataset = test_dataset.repeat(3)
test_dataset = test_dataset.batch(512)    # 아래 model.fit()에서의 batch_size 하이퍼파라미터와 동일



In [ ]:

# 모델을 계속해서 학습시킵니다.
# 이때, 기존의 모델의 가중치를 그대로 사용하므로 fine-tuning이라고도 합니다.
model.fit(train_aug_dataset, validation_data=test_dataset, epochs=100, batch_size=512)

# 학습이 완료된 모델을 저장합니다.
model.save('/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/output/your_updated_model_5.h5')

Epoch 1/100
7/7 [==============================] - 89s 2s/step - loss: 6.9247 - accuracy: 0.1211 - val_loss: 2.0274 - val_accuracy: 0.3263
Epoch 2/100
7/7 [==============================] - 7s 1s/step - loss: 2.6886 - accuracy: 0.1715 - val_loss: 2.2056 - val_accuracy: 0.2406
Epoch 3/100
7/7 [==============================] - 7s 1s/step - loss: 2.2996 - accuracy: 0.2038 - val_loss: 3.7092 - val_accuracy: 0.2073
Epoch 4/100
7/7 [==============================] - 7s 1s/step - loss: 2.1136 - accuracy: 0.3023 - val_loss: 2.4754 - val_accuracy: 0.2300
Epoch 5/100
7/7 [==============================] - 7s 1s/step - loss: 1.9850 - accuracy: 0.3224 - val_loss: 2.7486 - val_accuracy: 0.2322
Epoch 6/100
7/7 [==============================] - 7s 1s/step - loss: 1.6972 - accuracy: 0.3986 - val_loss: 4.0723 - val_accuracy: 0.2025
Epoch 7/100
7/7 [==============================] - 7s 1s/step - loss: 1.4042 - accuracy: 0.5100 - val_loss: 3.3554 - val_accuracy: 0.2265
Epoch 8/100
7/7 [================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
